In [1]:
import pandas as pd
from utils.config import FOLDER_DATA
from file_management import LIVE_PROCESSED_TABLE

# The most up to date scrapped csv version is used
test = pd.read_csv(f'{FOLDER_DATA}/{LIVE_PROCESSED_TABLE}') if LIVE_PROCESSED_TABLE else None

scrapped_films_202305171705.csv
processed_films_202305171705.csv


In [ ]:
import pandas as pd
import yaml

# Load the config.yaml file
with open('modeling/config.yaml') as file:
    cfg = yaml.safe_load(file)


df = pd.read_csv(f'{FOLDER_DATA}/{LIVE_PROCESSED_TABLE}') if LIVE_PROCESSED_TABLE else None

# Apply lambda function to each feature specified in config.yaml
for feature in cfg['model']['content_features']:
    if feature == 'synopsis':
        df[feature] = df[feature].apply(lambda x:x.split())
    else:
        pass

for feature in cfg['model']['processed_features']:
    df[feature] = df[feature].apply(lambda x: ','.join(map(lambda i: i.replace(' ', ''), x.split(','))))

# Display the modified DataFrame
df.head()

In [ ]:
# TODO: 

##### Temp solution

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load the dataset
dataset = pd.pd.read_csv(f'{FOLDER_DATA}/{LIVE_PROCESSED_TABLE}') if LIVE_PROCESSED_TABLE else None

# Create a TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Replace missing values with empty strings
dataset['synopsis'] = dataset['synopsis'].fillna('')

# Apply TF-IDF vectorizer to the 'synopsis' feature
tfidf_matrix = tfidf.fit_transform(dataset['synopsis'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get movie recommendations
def get_recommendations(title, cosine_sim_matrix, dataset):
    # Get the index of the movie that matches the title
    idx = dataset[dataset['title'] == title].index[0]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top 5 similar movies
    top_movies = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [movie[0] for movie in top_movies]

    # Return the top 5 similar movies
    return dataset['title'].iloc[movie_indices]

In [ ]:
# Example usage
movie_title = 'Forrest Gump'
recommendations = get_recommendations(movie_title, cosine_sim, dataset)
print(recommendations)